## Search and Download VIIRS Active Fire using 'earthaccess'

In [1]:
"""
Package imports and environment variables
"""

# Import modules and env vars

import os, time, shutil
import folium
import earthaccess
import warnings
import folium.plugins
import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import rasterio as rio
import numpy as np
import math

from rasterio.plot import show
from branca.element import Figure
from IPython.display import display
from shapely import geometry
from skimage import io
from datetime import timedelta
from shapely.geometry.polygon import orient
from matplotlib import pyplot as plt

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'

# Projection information
geog = 'EPSG:4326'  # Geographic projection
proj = 'EPSG:5070'  # Projected coordinate system

print("Success!")

Success!


In [2]:
def get_shapely_object(result:earthaccess.results.DataGranule):
    """
    Retrieve geospatial information from ECOSTRESS granule footprints.
    This function allows us to retrieve the geographic coverage for each granule and plot it on a map.
    
    :param 'result:earthaccess.results.DataGranule': a single data granule from earthaccess data search
    """
    
    # Get Geometry Keys
    geo = result['umm']['SpatialExtent']['HorizontalSpatialDomain']['Geometry']
    keys = geo.keys()

    if 'BoundingRectangles' in keys:
        bounding_rectangle = geo['BoundingRectangles'][0]
        # Create bbox tuple
        bbox_coords = (bounding_rectangle['WestBoundingCoordinate'],bounding_rectangle['SouthBoundingCoordinate'],
                       bounding_rectangle['EastBoundingCoordinate'],bounding_rectangle['NorthBoundingCoordinate'])
        # Create shapely geometry from bbox
        shape = geometry.box(*bbox_coords, ccw=True)
    elif 'GPolygons' in keys:
        points = geo['GPolygons'][0]['Boundary']['Points']
        # Create shapely geometry from polygons
        shape = geometry.Polygon([[p['Longitude'],p['Latitude']] for p in points])
    else:
         raise ValueError('Provided result does not contain bounding boxes/polygons or is incompatible.')
    return(shape)


# Convert bounding coordinates to Folium-ready data for mapping
def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation
    Leaflet interactive maps require a specific format for coordinates, this function sets that up for a given bounding box.

    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))
    
print("Ready")

Ready


In [3]:
# Load some data
fires = gpd.read_file(os.path.join(maindir,'aspen-fire/Aim2/data/spatial/mod/FIRED/fired_events_west_aspen.gpkg'))
print(fires.columns)

Index(['fired_id', 'ig_date', 'ig_day', 'ig_month', 'ig_year', 'last_date',
       'event_dur', 'tot_pix', 'tot_ar_km2', 'fsr_px_dy', 'fsr_km2_dy',
       'mx_grw_px', 'mn_grw_px', 'mu_grw_px', 'mx_grw_km2', 'mn_grw_km2',
       'mu_grw_km2', 'mx_grw_dte', 'x', 'y', 'ig_utm_x', 'ig_utm_y', 'lc_code',
       'lc_mode', 'lc_name', 'lc_desc', 'lc_type', 'eco_mode', 'eco_name',
       'eco_type', 'tot_perim', 'pct_aspen', 'geometry'],
      dtype='object')


#### Identify available VIIRS active fire products (VNP14)

In [17]:
# Search for VIIRS data collections
Query = earthaccess.collection_query().keyword('VNP14')
print(f'Collections found: {Query.hits()}')

# Return search results as a list
collections = Query.fields(['ShortName']).get(Query.hits())

# Retrieve Collection short-names
print("All available data products: ")
[product['short-name'] for product in [collection.summary() for collection in collections]]

Collections found: 11
All available data products: 


['VNP14A1',
 'VNP14A1',
 'VNP14',
 'VNP03MODLL',
 'VJ114A1',
 'VJ114',
 'VJ114IMG',
 'VNP14',
 'VNP14IMG',
 'VNP14_NRT',
 'VNP03MODLL']

In [18]:
# Do a test search
fire = fires[fires['fired_id'] == "42306"]  # Williams Fork Fire "45811.0"
# Retrieve the date part
ig_date = fire['ig_date']
last_date = fire['last_date']

# Define some search parameters
short_names = ['VNP14IMG']

# Define the date range
date_range = (ig_date.iloc[0], last_date.iloc[0])
print(date_range)

# Retrieve the coordinate pairs for the region of interest
perim = fire.to_crs(geog) # convert back to geographic coordinates
geom = perim.unary_union.envelope # dissolve into a single shape and get the envelope
coords = list(geom.exterior.coords) # retrieve the coordinate pairs

# Search for ECOSTRESS products matching our short names
result = earthaccess.search_data(
    short_name=short_names,
    polygon=coords,
    temporal=date_range,
    count=500, 
)

('2020-08-13 00:00:00', '2020-09-03 00:00:00')
Granules found: 64


Now let's grab the granule information and plot it.

In [20]:
# Save the results as a data frame, append to dictionary
df = pd.json_normalize(result)
# Create shapely polygons for result
geometries = [get_shapely_object(result[index]) for index in df.index.to_list()]
# Convert to GeoDataframe
results_gdf = gpd.GeoDataFrame(df, geometry=geometries, crs="EPSG:4326")
# Ensure 'start_datetime' and other relevant datetime columns are strings
results_gdf['start_datetime'] = results_gdf['umm.TemporalExtent.RangeDateTime.BeginningDateTime'].astype(str)
print(results_gdf.columns)

Index(['size', 'meta.concept-type', 'meta.concept-id', 'meta.revision-id',
       'meta.native-id', 'meta.collection-concept-id', 'meta.provider-id',
       'meta.format', 'meta.revision-date',
       'umm.TemporalExtent.RangeDateTime.BeginningDateTime',
       'umm.TemporalExtent.RangeDateTime.EndingDateTime', 'umm.GranuleUR',
       'umm.AdditionalAttributes',
       'umm.SpatialExtent.HorizontalSpatialDomain.Geometry.GPolygons',
       'umm.ProviderDates', 'umm.CollectionReference.ShortName',
       'umm.CollectionReference.Version', 'umm.PGEVersionClass.PGEVersion',
       'umm.RelatedUrls', 'umm.Projects', 'umm.DataGranule.DayNightFlag',
       'umm.DataGranule.Identifiers', 'umm.DataGranule.ProductionDateTime',
       'umm.DataGranule.ArchiveAndDistributionInformation', 'umm.Platforms',
       'umm.MetadataSpecification.URL', 'umm.MetadataSpecification.Name',
       'umm.MetadataSpecification.Version', 'geometry', 'start_datetime'],
      dtype='object')


In [21]:
# Create the interactive map using folium
fig = Figure(width="750px", height="375px")
map1 = folium.Map(tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}', attr='Google')
fig.add_child(map1)

fire_ = fire[['fired_id','geometry']]

# Plot Region of Interest
fire_.explore(
    popup=False,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="South Platte River Basin",
    m=map1
)

results_gdf[['meta.native-id','start_datetime','geometry']].explore(
    "meta.native-id",
    categorical=True,
    tooltip=[
        "meta.native-id",
        "start_datetime",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="EMIT",
    m=map1,
    legend=False
)

map1.fit_bounds(bounds=convert_bounds(results_gdf.unary_union.bounds))
map1.add_child(folium.LayerControl())
display(fig)

del fire_

In [8]:
results_urls = [granule.data_links() for granule in result]
results_urls[0:5]

[['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/VJ114IMG.002/VJ114IMG.A2020226.0800.002.2024070113843/VJ114IMG.A2020226.0800.002.2024070113843.nc'],
 ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/VNP14IMG.002/VNP14IMG.A2020226.0848.002.2024066004717/VNP14IMG.A2020226.0848.002.2024066004717.nc'],
 ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/VJ114IMG.002/VJ114IMG.A2020226.0936.002.2024070113845/VJ114IMG.A2020226.0936.002.2024070113845.nc'],
 ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/VJ114IMG.002/VJ114IMG.A2020226.1918.002.2024070113846/VJ114IMG.A2020226.1918.002.2024070113846.nc'],
 ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/VJ114IMG.002/VJ114IMG.A2020226.1924.002.2024070113841/VJ114IMG.A2020226.1924.002.2024070113841.nc']]

In [22]:
t0 = time.time()

dest = os.path.join(maindir,'aspen-fire/Aim2/data/spatial/raw/VNP14IMG/')
if not os.path.exists(dest):
    os.makedirs(dest)
    
# Retrieve granule asset ID from URL (to maintain existing naming convention)
for url in results_urls:
    granule_id = os.path.basename(url[0])
    print(f"Granule ID: {granule_id}")
    
    fp = os.path.join(dest,f'{granule_id}')

    # Download the Granule Asset if it doesn't exist
    if not os.path.isfile(fp):
        earthaccess.download(url[0], local_path=fp)
    else:
        print("*** Already downloaded ***")
        print(" ")

    print('Time to complete granule:', time.time() - t0)

Granule ID: VJ114IMG.A2020226.0800.002.2024070113843.nc


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Time to complete granule: 2.756728172302246
Granule ID: VNP14IMG.A2020226.0848.002.2024066004717.nc


...

Time to complete granule: 335.4778881072998
Granule ID: VNP14IMG.A2020246.2036.002.2024066052018.nc


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Time to complete granule: 337.88052320480347
